# Neural Networks in Compyute

In [ ]:
import compyute as cp

## Prepare the Data

`Compyute` offers some helper functions to make data preparation easy. Here we use `split_train_val_test` to split the data into train, val, and test sets. We use `normalize` to normalize the features.

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd
from compyute.preprocessing import split_train_val_test, normalize

# download the data
url = "https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv"
data = pd.read_csv(url)

# encode the targets
data["variety"] = data["variety"].astype("category").cat.codes
data_tensor = cp.tensor(data.to_numpy())

# split the data into train, val, test
train, val, test = split_train_val_test(data_tensor, ratio_val=0.25, ratio_test=0.25)

# split features from targets
X_train, y_train = train[:, :-1], train[:, -1].to_int()
X_val, y_val = val[:, :-1], val[:, -1].to_int()
X_test, y_test = test[:, :-1], test[:, -1].to_int()

# normalize features
X_train = normalize(X_train, dim=0, low=-1, high=1)
X_val = normalize(X_val, dim=0, low=-1, high=1)
X_test = normalize(X_test, dim=0, low=-1, high=1)

print (f'{X_train.shape=}')
print (f'{y_train.shape=}')
print (f'{X_val.shape=}')
print (f'{y_val.shape=}')
print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

## Build the Neural Network Structure
Here the individual layers of the neural network models are defined.

In [ ]:
from compyute import nn

model = nn.Sequential(
    nn.Linear(in_channels=4, out_channels=16),  # 4 input nodes, 16 neurons
    nn.ReLU(),  # activation function
    nn.Linear(in_channels=16, out_channels=3),  # 16 input nodes, 3 neurons
)

In [ ]:
summary = cp.nn.utils.get_module_summary(model, input_shape=(4,))
print(summary)

## Train the Model

To train a model, the easiest way is to use the `Trainer` object. Here we use `sgd` as an optimizer, `cross_entropy` as a loss function, and `accuracy` as a metric. Furthermore, we use the `History` callback to track the loss and accuracy values during training. We also use the `ProgressBar` callback to track the progress of the training process.

Note: If you want to customize e.g. the optimizer with a different learning rate, instead of defining the optimizer via a string, you can also pass an optimizer object.

In [ ]:
from compyute.nn.trainer import Trainer
from compyute.nn.trainer.callbacks import History, ProgressBar

history = History()
prograss = ProgressBar(mode="epoch")

trainer = Trainer(
    model=model,
    optimizer="sgd",  # alternatively, pass in an object, e.g. nn.optimizers.SGD()
    loss="cross_entropy",
    metric="accuracy",
    callbacks=[history, prograss]
)

In [ ]:
trainer.train(X_train, y_train, epochs=10000, batch_size=256, val_data=(X_val, y_val))

Plot the training history

In [ ]:
# ! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

def plot_history(t1, t2):
    trace1 = history[t1]
    trace2 = history[t2]
    plt.figure(figsize=(10, 3))
    plt.plot(cp.arange(start=1, stop=len(trace1) + 1), trace1, linewidth=1, label=t1)
    plt.plot(cp.arange(start=1, stop=len(trace2) + 1), trace2, linewidth=1, label=t2)
    plt.xlabel("step")
    plt.ylabel("loss/accuracy")
    plt.legend()

plot_history("loss", "accuracy_score");
plot_history("val_loss", "val_accuracy_score");

## Evaluate the Model

Using the defined metric, the model's performance can be evaluated using testing/validation data.

In [ ]:
loss, accuracy = trainer.evaluate_model(X_test, y_test)
print(f'loss {loss:.4f}')
print(f'accuracy {100 * accuracy:.2f}')

## Save the trained Model

Models are saved by serializing the state dictionary of the model and the optimizer.

In [ ]:
state = {
    "model": model.get_state_dict(),
    "optimizer": trainer.optimizer.get_state_dict()
}
cp.save(state, "iris_model.cp")

To load a saved model, you have to instanciate a new model and load the state dictionary.

In [ ]:
from compyute import nn

# create a model and optimizer instance
loaded_model = nn.Sequential(
    nn.Linear(in_channels=4, out_channels=16),
    nn.ReLU(),
    nn.Linear(in_channels=16, out_channels=3),
)
loaded_optimizer = nn.optimizers.SGD(loaded_model.get_parameters())

# load parameters from the checkpoint
loaded_state = cp.load("iris_model.cp")
loaded_model.load_state_dict(loaded_state["model"])
loaded_optimizer.load_state_dict(loaded_state["optimizer"])

# test the loaded model
sample = cp.tensor([[5.1, 3.5, 1.4, 0.2]])
loaded_model(sample)